# Perfilamiento del Código

## Librerías, funciones y datos para las pruebas

In [1]:
%pip install -q --user line_profiler
%pip install -q --user memory_profiler
%pip install -q --user psutil
%pip install -q --user guppy3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
import numpy as np
import pandas as pd
import os
import time

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [3]:
import math
from scipy.integrate import quad

In [4]:
# Importamos el módulo con nuestras funciones
import sys
sys.path.append('./../../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#Own Library modules
import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.pca import todoJunto
from src.test_algorithms import comparaModelos
from src.test_algorithms.err_relativo import err_relativo

In [8]:
df = pd.read_csv('../../data/nndb_flat.csv', encoding = "UTF-8")

In [9]:
df.drop(df.columns[df.columns.str.contains('_USRDA')].values, 
        inplace=True, axis=1)
df = df.drop(columns=['ID','FoodGroup','ShortDescrip','Descrip','CommonName','MfgName','ScientificName'])

In [10]:
####Se estandarizan los datos.
scaler = preprocessing.StandardScaler()
df_nutrientes = scaler.fit_transform(df)

# Pruebas de Tiempos

El objetivo de este notebook es perfilar y los resultados de PCA de cada una de nuestras funciones:
+ PCA usando la función SVD de numpy.
+ PCA a partir del algoritmo QR
+ PCA a partir del método de la potencia

## 1. A través de time

In [ ]:
start_time1 = time.time()
todoJunto.PCA_from_sklearn(df_nutrientes)
end_time1 = time.time()

start_time2 = time.time()
todoJunto.PCA_from_SVD(df_nutrientes)
end_time2 = time.time()

start_time3 = time.time()
todoJunto.PCA_from_QR_vf(df_nutrientes)
end_time3 = time.time()

start_time4 = time.time()
todoJunto.PCA_from_potencia(df_nutrientes)
end_time4 = time.time()

In [18]:
secs1 = end_time1-start_time1
secs2 = end_time2-start_time2
secs3 = end_time3-start_time3
secs4 = end_time4-start_time4
print("PCA_from_sklearn tomó",secs1,"segundos" )
print("PCA_from_SVD tomó",secs2,"segundos" )
print("PCA_from_QR tomó",secs3,"segundos" )
print("PCA_from_potencia tomó",secs4,"segundos" )

PCA_from_sklearn tomó 0.020970582962036133 segundos
PCA_from_SVD tomó 0.007998943328857422 segundos
PCA_from_QR tomó 1.156999111175537 segundos
PCA_from_potencia tomó 28.687139749526978 segundos


Las funciones provenientes de sklearn y numpy fueron claramente más rápidas que las que elaboramos para el proyecto.

## 2. A través de %timeit

Al igual que en las notas, se utilizará un loop de tamaño $5$ y $20$ repeticiones.

In [29]:
%timeit -n 5 -r 20 todoJunto.PCA_from_sklearn(df_nutrientes)

15.4 ms ± 506 µs per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

In [32]:
%timeit -n 5 -r 20 todoJunto.PCA_from_SVD(df_nutrientes)

6.42 ms ± 483 µs per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

In [33]:
%timeit -n 5 -r 20 todoJunto.PCA_from_QR_vf(df_nutrientes)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\numeric.py:906: RuntimeWarning: overflow encountered in multiply
  return multiply(a.ravel()[:, newaxis], b.ravel()[newaxis, :], out)
./../..\src\pca\funciones_examen_QR.py:143: RuntimeWarning: invalid value encountered in multiply
  Arv[j:m,j:n]=Arv[j:m,j:n]-beta*np.outer(v,(np.transpose(v)@Arv[j:m,j:n]))
./../..\src\pca\funciones_examen_QR.py:234: RuntimeWarning: invalid value encountered in multiply
  Qj[(j-1):m,(j-1):m]=np.eye(m-(j-1))-beta*np.outer(v,v)
./../..\src\pca\todoJunto.py:190: RuntimeWarning: invalid value encountered in less
  conteo = (varianza_acumulada)  <  0.8


1.17 s ± 78.9 ms per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

In [34]:
%timeit -n 5 -r 20 todoJunto.PCA_from_potencia(df_nutrientes)

406 ms ± 20.4 ms per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

# Pruebas de uso de CPU